
A regressor object is built based on the (multi-fidelity) data and the design.

This type of multi-fidelity Gaussian process regression is described in the following article:

```{bibliography}
Wu, J.; Toscano-Palmerin, S.; Frazier, P. I. & Wilson, A. G.
Practical multi-fidelity bayesian optimization for hyperparameter tuning 
Uncertainty in Artificial Intelligence, 2020, 788-798
```

In [ ]:
regressor = f3dasm.regression.gpr.Stmf(
    mf_train_data=mf_train_data[-1],
    mf_design=mf_train_data[-1].design,
    noise_fix=noise_fix,
)

The resulting surrogate model is obtained by training the regressor.

In [ ]:
surrogate = regressor.train()

This surrogate model can then be used to predict the (high-fidelity) output for any point in the design space.

In [ ]:
test_sampler = f3dasm.sampling.LatinHypercube(design=mf_design_space[-1])
test_data = test_sampler.get_samples(numsamples=500)

mean, var = surrogate.predict(test_data)

Finally, let's plot the prediction alongside the exact objective landscape.

In [ ]:
test_data.add_output(mean)
test_data_var = np.hstack((test_data.data.values, var))
test_sort = test_data_var[test_data_var[:, 0].argsort()]

ucb, lcb = [test_sort[:, 2] + 2 * (-1) ** k * np.sqrt(np.abs(test_sort[:, 3])) for k in range(2)]

import matplotlib.pyplot as plt
hf_data = train_data.data.loc[train_data.data['input', 'fid'] == 1.]
plt.scatter(hf_data['input', 'x0'], hf_data['output'], label='High fidelity data', c='b')
plt.plot(test_sort[:, 0], test_sort[:, 2], color='purple', label='High fidelity prediction')
plt.fill_between(test_sort[:, 0].flatten(), lcb, ucb, color='purple', alpha=.25, label='Confidence')
plt.plot(test_sort[:, 0], base_fun(test_sort[:, 0][:, None]), 'b--', label='High fidelity exact')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()